In [1]:
##_____________________________________________Paquetes necesarios________________________________________________##

import zipfile # Descomprimir archivo .zip
from pathlib import Path # Rutas
import csv # Archivos csv
import math # Logaritmo
import copy # Copiar datos y sus rutas

###____________________________________Descomprimir archivos y localizarlos_______________________________________###

def descomprimir_archivos(archivo_zip, directorio_destino): # Descomprimo los archivos y los guardo 
    with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:      # en directorio destino.   
        zip_ref.extractall(directorio_destino)

def obtener_rutas_archivos(direct_principal):
    # Luego guardo la ruta de cada archivo en una lista, buscandolos con rglob y diciendole las características.
    return list(direct_principal.rglob("RUN*/experimento*.csv")) 

# Estos datos son fijos
archivo_zip = '/home/alumno01/SBIO/practicas_programacion/practica_1/runs.zip'
directorio_destino = '/home/alumno01/SBIO/practicas_programacion/practica_1/runs'    
direct_principal = Path('/home/alumno01/SBIO/practicas_programacion/practica_1/runs/')

# Ejecuto la función descomprimir y guardo las rutas de los archivos en lista_rutas_archivos.
descomprimir_archivos(archivo_zip, directorio_destino)
lista_rutas_archivos = obtener_rutas_archivos(direct_principal)

###_____________________________________Cargar datos en una estructura de datos___________________________________###

# Esta función toma como argumento lista_rutas_archivos, generada por la anterior función.
def cargar_datos(lista_rutas_archivos):
    lista_RNA_Seq = [] # Creo una lista local en la que ir guardando un diccionario para cada RUN.
    
    for i, dato_run in enumerate(lista_rutas_archivos, start=0): # Vamos recorriendo todas las rutas 
        # para ir abriendo los archivos 1 a 1.
        RNA_Seq = {} # Creo un diccionario en el que voy a ir guardando los datos por RUN.
        
        with open(dato_run, mode='r') as csv_file:
           # El primer problema es que no todos los ficheros usan los mismos delimitadores.
            linea = csv_file.readline()
            # Abro solo la primera línea del fichero csv, y le digo que si en esa línea encuentra  un ';', 
            # entonces delim=';', y así con todos los separadores que he definido.
            for delim in linea:
                if ';' in linea:
                    delim = ';'
                elif '\t' in linea:
                    delim = '\t'
                elif '|' in linea:
                    delim = '|'
                elif ':' in linea:
                    delim = ':'
                else:
                # Si el archivo no usa uno de los separadores definidos más comunes, 
                # dará un mensaje de error.
                    print("El archivo tiene un separador no valido para el programa, cambielo.")
           
            # Volvemos a la primera línea del archivo csv para no perder la cabecera.        
            csv_file.seek(0) 
            
            # Creamos la variable del archivo leido, determinando el delimitador, y lo
            #convertimos en una lista.
            csv_reader = csv.reader(csv_file, delimiter=delim)
            lista_csv_reader = list(csv_reader)
            
            # La primera posición de mi lista csv_reader será el encabezado.
            encabezado = lista_csv_reader[0]
            
            # Las siguientes, de la 1 hasta el final, serán los datos de cada gen para cada muestra.
            datos = lista_csv_reader[1:]
            
            # El número de genes para cada RUN será la longitud de la lista datos.
            genes = len(datos)
            
            # Cada fila datos se agrega al diccionario datos_expresion, usando el valor de la columna clave 
            # como clave del diccionario, que se corresponde a un determinado gen para cada fila.
            datos_expresion = {}
            for fila in datos:
                gen = fila[0]  
                datos_expresion[gen] = fila[1:]

            # Vamos creando los diccionarios para cada RUN a medida que se va recorriendo la lista_rutas_datos
            RNA_Seq[f"RUN{i + 1}"]={'muestras':encabezado[1:], 'genes':genes, 'datos':datos_expresion}
            
        # Vamos guardando el diccionario de cada RUN en la lista_RNA_Seq
        lista_RNA_Seq.append(RNA_Seq)
    
    
    # Esta parte la tuve que hacer cuando me di cuenta que la estructura de mis datos no era la óptima para
    # poder resolver los demás ejercicios.
    # Aquí creo un nuevo diccionario, en el que tengo todos los daatos de todos los RUN. Empiezo creándolo
    # copiando por completo, el diccionario del último RUN. https://docs.python.org/3/library/copy.html
    RNA_Seq_todos = {'todos': copy.deepcopy(RNA_Seq['RUN4']) }
    
    # Recorro lista_RNA_Seq, y voy sacando los datos de cada RUN y los meto en el diccionario RNA_Seq_todos.
    i=1
    for RNA_Seq in lista_RNA_Seq[0:3]:
        # Guardo en una clave muestras una lista con todas los elementos de la lista 'muestras' de cada RUN.
        RNA_Seq_todos['todos']['muestras'] += RNA_Seq[f"RUN{i}"]['muestras']
        # Hago lo mismo para el diccionario datos_expresion, que tiene la clave 'datos', pero al ser un diccionario
        # tengo que recorrerlo. 
        for gen, expresion in RNA_Seq[f"RUN{i}"]['datos'].items():
            RNA_Seq_todos['todos']['datos'][gen] += expresion
        i +=1
            
    return lista_RNA_Seq, RNA_Seq_todos


###____________________Calcular número de muestras por tipo y por RUN y el número de genes_______________________###

# Esta función toma como argumento lista_RNA_Seq, la recorre, y añade 1 al contador de WT o KO conforme se
# encuentr un WT o KO en la lista muestras de cada RUN. Esa información luego la añade a un diccionario llamado
# info_por_run.

def muestras_por_tipo_y_run(lista_RNA_Seq):
    info_por_run = {}
    info_por_run.clear()  # Limpiar el diccionario antes de agregar nuevos valores
    for RNA_Seq in lista_RNA_Seq:
        for run, data in RNA_Seq.items():
            muestras = data.get('muestras')
            num_WT = sum(1 for muestra in muestras if muestra.startswith('WT'))
            num_KO = sum(1 for muestra in muestras if muestra.startswith('KO'))
            numero_genes = data.get('genes')
            info_por_run[run] = {'Número de genes': numero_genes, 
                         'Número de muestras WT': num_WT, 
                         'Número de muestras KO': num_KO}
    return info_por_run

###__________________________________Calcular número total de muestras WT y KO____________________________________###

# Esta función toma como argumento lista_RNA_Seq, la recorre, y añade 1 al contador de WT o KO conforme se
# encuentr un WT o KO en la lista muestras de cada RUN. Sumamos todos los componentes de cada lista, y obtenemos
# el total de muestras KO y WT.

def muestras_totales_tipo(lista_RNA_Seq):
    info_WT = []
    info_KO = []
    info_WT.clear()
    info_KO.clear()
    for RNA_Seq in lista_RNA_Seq:
        for run, data in RNA_Seq.items():
            muestras = data.get('muestras')
            num_WT = sum(1 for muestra in muestras if muestra.startswith('WT'))
            num_KO = sum(1 for muestra in muestras if muestra.startswith('KO'))
            info_WT.append(num_WT)
            info_KO.append(num_KO)


    total_WT = sum(info_WT)
    total_KO = sum(info_KO)
    numero_total_muestras_tipo = {'Número total de muestras WT': total_WT, 
                                  'Número total de muestras KO': total_KO}

    return numero_total_muestras_tipo

###___________________________________________Procesador de RUNs _________________________________________________###
 
# Esta función toma como argumento los RNA_Seq, los recorre. Calcula el número de muestras WT, KO y totales,
# al igual que la función anterior, pero para cada gen calcula la expresión media para WT, K0 y totales, para luego 
# sacar diferentes estadísticos y guardarlos en un diccionario.

def procesador_datos_run(RNA_Seq):     
    info_WT = []
    info_KO = []
    info_total = []

    for run, data in RNA_Seq.items():
       
        expresiones = data.get('datos')
        muestras = data.get('muestras')

        num_muestras_KO = sum(1 for muestra in muestras if muestra.startswith('KO'))
        num_muestras_WT = sum(1 for muestra in muestras if muestra.startswith('WT'))
        num_muestras_total = num_muestras_KO + num_muestras_WT

        info_WT.append(num_muestras_WT)
        info_KO.append(num_muestras_KO)
        info_total.append(num_muestras_total)

        resultados_KO = {}
        resultados_WT = {}
        resultados_total = {}

        for gen, expresion in expresiones.items():
            # Como expresiones es un diccionario, y además está gardado como strings, sobreescribimos
            # la variable expresion in expresiones para que pueda trabajar con ellas.
            expresion = list(map(int, expresion))
            
            # Tengo separadas las muestras de las expresiones para cada gen, con lo cual en principio
            # no sé a qué muestra se corresponde cada valor de expresión.
            
            # Para ello, conforme voy recorriendo expresion in expresiones, la posición de expresion
            # se corresponde con la posición de muestra in muestras cuaando muestra empieza por WT o KO.

            expresion_WT = [expresion[i] for i, muestra in enumerate(muestras) if muestra.startswith('WT')]
            media_total_WT = round(sum(expresion_WT) / num_muestras_WT, 3)
            resultados_WT[gen] = media_total_WT

            expresion_KO = [expresion[i] for i, muestra in enumerate(muestras) if muestra.startswith('KO')]
            media_total_KO = round(sum(expresion_KO) / num_muestras_KO, 3)
            resultados_KO[gen] = media_total_KO

            expresion_total = [sum(expresion)]
            media_total = round(sum(expresion_total) / num_muestras_total, 3)
            resultados_total[gen] = media_total
            
        # Obtener los 10 genes más expresados en su valor medio en muestras WT para un RUN
        top_10_WT = sorted(resultados_WT.items(), key=lambda x: x[1], reverse=True)[:10]
        
        # Obtener los 10 genes menos expresados en su valor medio en muestras WT para un RUN
        bottom_10_WT = sorted(resultados_WT.items(), key=lambda x: x[1], reverse=False)[:10]
        
        # Obtener los 10 genes más expresados en su valor medio en muestras KO para un RUN
        top_10_KO = sorted(resultados_KO.items(), key=lambda x: x[1], reverse=True)[:10]
        
        # Obtener los 10 genes menos expresados en su valor medio en muestras KO para un RUN
        bottom_10_KO = sorted(resultados_KO.items(), key=lambda x: x[1], reverse=False)[:10]  
        
         # Obtener los 10 genes más expresados en su valor medio en muestras TOTAL para un RUN
        top_10_total = sorted(resultados_total.items(), key=lambda x: x[1], reverse=True)[:10]
        
        # Obtener los 10 genes menos expresados en su valor medio en muestras TOTAL para un RUN
        bottom_10_total = sorted(resultados_total.items(), key=lambda x: x[1], reverse=False)[:10]
        
        # Guardo todos esos cálculos en un diccionario.
        resultados_por_RUN={
            'Top 10 genes más expresados': top_10_total,
            'Bottom 10 genes menos expresados': bottom_10_total,
            'Top 10 genes más expresados en muestras WT': top_10_WT,
            'Bottom 10 genes menos expresados en muestras WT': bottom_10_WT,
            'Top 10 genes más expresados en muestras KO': top_10_KO,
            'Bottom 10 genes menos expresados en muestras KO': bottom_10_KO
        }
     
    # Me devuelve argumentos que necesitaré posteriormente, como son resultados_WT y resultados_KO
    return resultados_por_RUN, resultados_WT, resultados_KO

###__________________________________________Función Calcular Fold Change_________________________________________###

# Esta función tiene como argumentos de entrada los diccionarios resultados_KO y resultados_WT, resultados de la
# función anterior, y recorre los diccionarios y calcula e fold_change para cada gen. Solo se ejecutará cuando
# se le pidan los datos para todos los RUN.

def fold_change(resultados_KO, resultados_WT):
    log_fold_change = {}
    for gen in resultados_KO:
        fold_change = resultados_KO[gen] / resultados_WT[gen]
        log_fold_change[gen] = round(abs(math.log(fold_change, 2)), 3) 
    
    # Calculo el máximo y el mínimo valor del diccionario para poder establecer un rango de valores al usuario en 
    # el menú.
    mayor_log_fc = max(log_fold_change.values())
    menor_log_fc = min(log_fold_change.values())
    
    return log_fold_change, mayor_log_fc, menor_log_fc

###_____________________________________Función Separar genes por Fold Change_____________________________________###

# Esta función toma como argumentos de entrada el diccionario construido en la función anterior y un número
# i, que es dado por el usuario en el menú, y que será el numero a partir el cual devidir los genes sobre expresados
# y los infraexpresados, los cuales se guardan en dos listas diferentes. 

def separar_genes_por_fc(log_fold_change, i):
    up_regulated_genes = []
    down_regulated_genes = []

    for gen, log_fc in log_fold_change.items():
        if log_fc < i:
            down_regulated_genes.append(gen)
        else:
            up_regulated_genes.append(gen)

    return up_regulated_genes, down_regulated_genes


###___________________________________________Menú de consulta del programa___________________________________###

def main():
    # Cargamos los datos desde los archivos
    estructura_datos_runs, estructura_todos_los_datos_runs = cargar_datos(lista_rutas_archivos)
    # Entrada del usuario. Menú principal del progrma
    while True:
        print("\n\n1. Listar RUNs")
        print("2. Total de muestras cargadas")
        print("3. Salir del programa")
        opcion = input("\nIndique la opción que desea ejecutar (1, 2 o 3): ")
        # Opción para listar RUNs
        if opcion == "1":
            # Obtener infomación sobre las muestras por tipo y RUN
            info_por_run = muestras_por_tipo_y_run(estructura_datos_runs)
            print(info_por_run)
            
            # Submenú para ver detalles de un RUN específico
            while True:
                run_seleccionado = input("\nIndique el número de RUN para ver detalles, pulse 'X' para ver los detalles de todo el experimento, o presione '0' para regresar al menú principal: ")
                if run_seleccionado == "0":
                    break
                # Procesar datos para un RUN específico y mostrar detalles cuando run_seleccionado es un número y está dentro de la longitud de estructura_datos_runs.
                elif run_seleccionado.isdigit() and int(run_seleccionado)<=len(estructura_datos_runs):
                    # Solo procesará el run seleccionado e imprimirá los resultados
                    resultado_por_RUN, resultados_WT, resultados_KO = procesador_datos_run(estructura_datos_runs[int(run_seleccionado)-1])
                    print(resultado_por_RUN)                    
                elif run_seleccionado.upper() == "X":
                    # Si el usuario ingresa una X, calculará los estadísticos para el experimento completo.
                    info_total_RUN, resultados_WT, resultados_KO = procesador_datos_run(estructura_todos_los_datos_runs)
                    print(info_total_RUN)
                    
                    # Submenu para el FOLD_CHANGE, que solo saldrá cuando el usuario pida estadísticos de todos los RUN juntos
                    while True:
                        respuesta = input("\n¿Quiere saber qué genes están sobreexpresados e infraexpresados atendiendo al |Log(Fold Change)|? N = No / S = Sí")
                        if respuesta == "N":
                            break
                        elif respuesta == "S":
                            while True:
                                # Necesito mayor_log_fc y menor_log_fc para poder darle al usuario el rango en el cual puede seleccionar un número de FC
                                log_fold_change, mayor_log_fc, menor_log_fc = fold_change(resultados_KO, resultados_WT)
                                print("Elija un valor entre", menor_log_fc, "y", mayor_log_fc, ".")                                
                                try:
                                    i = float(input("Introduzca un valor. Presione 0 para regresar al menú anterior:"))
                                    if i == 0:
                                        break
                                    # Si el numero introducido se encuentra en el rango, se ejecuta la función separar_genes_por_fc
                                    elif menor_log_fc <= i <= mayor_log_fc:
                                        up_reg_genes, down_reg_genes = separar_genes_por_fc(log_fold_change, i)
                                        print("Estos son los genes sobre-expresados:", up_reg_genes)
                                        print("Estos son los genes infra-expresados:", down_reg_genes)
                                        break
                                    else:
                                        print("El valor no está dentro del rango válido. Inténtelo de nuevo.")
                                except ValueError:
                                    print("Ingrese un número válido.")
                        else:
                            print("Tiene que seleccionar 0 (No) o 1 (Sí)")
                else:
                    print("Ingrese un número válido de RUN.")                    
        elif opcion == "2":
            numero_total_muestras_tipo = muestras_totales_tipo(estructura_datos_runs)
            print(numero_total_muestras_tipo)  
        elif opcion == "3":
            break
        else:
            #IMPORTANTE. Avisar al usuario cuando la entrada no es correcta
            print("La opción indicada no está disponible. Pruebe con la siguientes:\n")
    print("El programa ha finalizado correctamente.")
    
if __name__ == "__main__":
    main()



1. Listar RUNs
2. Total de muestras cargadas
3. Salir del programa



Indique la opción que desea ejecutar (1, 2 o 3):  1


{'RUN1': {'Número de genes': 485, 'Número de muestras WT': 3, 'Número de muestras KO': 4}, 'RUN2': {'Número de genes': 485, 'Número de muestras WT': 6, 'Número de muestras KO': 4}, 'RUN3': {'Número de genes': 485, 'Número de muestras WT': 4, 'Número de muestras KO': 5}, 'RUN4': {'Número de genes': 485, 'Número de muestras WT': 2, 'Número de muestras KO': 4}}



Indique el número de RUN para ver detalles, pulse 'X' para ver los detalles de todo el experimento, o presione '0' para regresar al menú principal:  1


{'Top 10 genes más expresados': [('GNAI1', 807.429), ('NRG1', 771.714), ('NCKIPSD', 764.0), ('PADI4', 731.571), ('ABCB6', 729.571), ('GAST', 727.286), ('DDX58', 720.429), ('ENG', 718.0), ('MMP9', 716.0), ('MMP12', 709.857)], 'Bottom 10 genes menos expresados': [('CYB5R3', 162.0), ('PICK1', 167.143), ('GLI1', 252.0), ('NR1I2', 262.286), ('MIR1246', 264.714), ('RBKS', 271.857), ('TGM4', 277.286), ('ARL2BP', 279.714), ('RIPK2', 281.857), ('ATXN1L', 284.286)], 'Top 10 genes más expresados en muestras WT': [('NRM', 933.0), ('AGTR1', 899.667), ('ADA2', 881.0), ('IL2RA', 866.667), ('IDO1', 859.667), ('MDFI', 853.0), ('CD69', 845.333), ('AHR', 839.333), ('F2', 837.0), ('EREG', 836.667)], 'Bottom 10 genes menos expresados en muestras WT': [('BAAT', 80.333), ('CYB5R3', 82.0), ('RHOA', 121.667), ('GPHA2', 138.667), ('IL15', 141.667), ('GLYAT', 155.0), ('TLR2', 160.0), ('TP63', 163.333), ('FLT1', 170.0), ('CXCL9', 177.0)], 'Top 10 genes más expresados en muestras KO': [('NRG1', 862.0), ('PARP9', 8

...

Estos son los genes sobre-expresados: ['IL1RN', 'AICDA', 'NEU1', 'VEGFA', 'NLRP3', 'FLT1', 'ISG15', 'ASPM', 'CERT1', 'BAAT', 'CLEC7A', 'SFTPD', 'CDC42', 'PPARA', 'STAT1', 'RELB', 'CASP1', 'DDX21', 'CYP2B6', 'ATP6V0A2', 'LBP', 'LIF', 'SAGE1', 'F2R', 'GPHA2', 'AMH', 'ARL2BP', 'CDH11', 'NRM', 'EHD1', 'SP2', 'TMEM37', 'CD274', 'SERPINC1', 'FUT2', 'GLI1', 'CDCA7L', 'EXOG', 'IL18', 'APC', 'TSPAN31', 'LAMP3', 'AKT1', 'CWC15', 'CD40LG', 'PLK2', 'CD79A', 'CYCSP39', 'CYB5R3', 'PAK3', 'PRKAA2', 'IL15', 'ADA2', 'SNAP91']
Estos son los genes infra-expresados: ['IL1A', 'ACE2', 'GPT', 'IL6', 'CRP', 'PARP9', 'GNAI1', 'CD8A', 'CD4', 'DEFA5', 'CD19', 'FCGR3A', 'NCAM1', 'IL2', 'IL4', 'IFNG', 'IL10', 'TNF', 'ATXN1L', 'TMPRSS2', 'TNNI3', 'ANXA13', 'SHBG', 'FGB', 'ERBB2', 'INS', 'ANGPT1', 'ANGPT2', 'ANGPT4', 'ANGPTL3', 'AGT', 'ACE', 'XYLT2', 'CXCL8', 'SLC4A2', 'BTK', 'REN', 'SLC5A2', 'BMPR2', 'ESR1', 'SEA', 'TLR2', 'TLR4', 'GH1', 'GGH', 'HSPA4', 'HSP90AA1', 'MIR1246', 'PLAT', 'ERG', 'ORF1', 'ALB', 'LAP', 'C


¿Quiere saber qué genes están sobreexpresados e infraexpresados atendiendo al |Log(Fold Change)|? N = No / S = Sí N

Indique el número de RUN para ver detalles, pulse 'X' para ver los detalles de todo el experimento, o presione '0' para regresar al menú principal:  0




1. Listar RUNs
2. Total de muestras cargadas
3. Salir del programa



Indique la opción que desea ejecutar (1, 2 o 3):  3


El programa ha finalizado correctamente.
